In [3]:
import ee
# Authenticate and initialize the Earth Engine API
ee.Authenticate()
ee.Initialize(project='ee-hungletatdac12345')

# Define the polygon around Canberra, Australia large enough for 8 rectangles

north = 19.9614
west = 105.7595
east = 105.9666
south = 19.7699

# Create the polygon coordinates
hoang_hoa_poly = [
    [north, west],  # Top-left
    [north, east],  # Top-right
    [south, east],  # Bottom-right
    [south, west],  # Bottom-left
    [north, west]   # Closing the polygon
]
hoang_hoa_poly=[[poly[1],poly[0]] for poly in hoang_hoa_poly]   
polygon = ee.Geometry.Polygon(hoang_hoa_poly)
# Parameters for the number of rectangles
numRectsX = 5  # Number of rectangles along the X axis
numRectsY = 5 # Number of rectangles along the Y axis

def maskS2clouds(image):
    qa = image.select('QA60')
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
             .And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask).divide(10000)

# Function to subdivide the polygon into rectangles based on the number of rectangles
def subdivide_polygon(polygon, numRectsX, numRectsY):
    bounds = polygon.bounds()
    coords = bounds.coordinates().get(0).getInfo()
    xMin = coords[0][0]
    xMax = coords[2][0]
    yMin = coords[0][1]
    yMax = coords[2][1]

    rectWidth = (xMax - xMin) / numRectsX
    rectHeight = (yMax - yMin) / numRectsY

    rectangles = []

    for x in range(numRectsX):
        for y in range(numRectsY):
            x0 = xMin + x * rectWidth
            y0 = yMin + y * rectHeight
            x1 = x0 + rectWidth
            y1 = y0 + rectHeight
            rect = ee.Geometry.Rectangle([x0, y0, x1, y1])
            rectangles.append(rect)

    return rectangles

# Call the function to create the rectangles
rectangles = subdivide_polygon(polygon, numRectsX, numRectsY)

# Function to export images to Google Drive
def prepare_export_tasks(rectangles):
    tasks = []
    for index, rect in enumerate(rectangles):
        image = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
            .filterBounds(rect) \
            .filterDate('2023-01-01', '2023-12-31') \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
                
        image=image.map(maskS2clouds)\
            .median() \
            .select(['B4', 'B3', 'B2', 'B8']) \
            .clip(rect)

        task = ee.batch.Export.image.toDrive(
            image=image,
            description='Image_' + str(index),
            folder='HoangHoaSubImages',
            scale=5,  # Adjust this scale as needed
            region=rect,
            maxPixels=1e9,
            fileFormat='GeoTIFF',
            formatOptions={'cloudOptimized': True}
        )
        tasks.append(task)
    
    return tasks

# Call the function to prepare the tasks
tasks = prepare_export_tasks(rectangles)

# Tasks are now prepared but not started
# You can start them later by calling task.start() on each task when you are ready


In [ ]:
len(tasks)

In [ ]:
for task in tasks:
    task.start()